In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Dataset
data = pd.read_csv("https://tinyurl.com/y2qmhfsr")

L = 0.05 # pace of learning

inputs = (data.iloc[:, 0:3].values / 255)
outputs = data.iloc[:, -1].values

# Training split
X_train, X_test, Y_train, Y_test = train_test_split(inputs, outputs, test_size=1/3)
n = X_train.shape[0]

In [3]:
# Creating Neural Network with random weights and bias
w_hidden = np.random.rand(3, 3)
w_output = np.random.rand(1, 3)

b_hidden = np.random.rand(3, 1)
b_output = np.random.rand(1, 1)

# Activation functions
relu = lambda x: np.maximum(x, 0)
sigmoid = lambda x: 1 / (1 + np.exp(-x))

# Forward pass
def forward_pass(X):
    Z1 = w_hidden @ X + b_hidden
    A1 = relu(Z1)
    Z2 = w_output @ A1 + b_output
    A2 = sigmoid(Z2)
    return Z1, A1, Z2, A2

In [121]:
# # PREPARING FOR BACKPROPAGATION
# # Searching derivatives weight and bias for cost function (C = A2 - Y)**2
# # where C is cost function, A2 is sigmoid output for neural network and Y is real value)

# # Derivatives in Sympy
# from sympy import *
# W1, W2, B1, B2, A1, A2, Z1, Z2, X, Y = symbols('W1 W2 B1 B2 A1 A2 Z1 Z2 X Y')

# # derivative of C (cost function) with respect to A2
# C = (A2 - Y)**2
# dC_dA2 = diff(C, A2)
# print("dC_dA2 =", dC_dA2)

# # derivative of A2 with respect to Z2
# sigmoid = lambda x: 1 / (1 + exp(-x))
# _A2 = sigmoid(Z2)
# dA2_dZ2 = diff(_A2, Z2)
# print("dA2_dZ2 =", dA2_dZ2)

# # derivative of Z2 with respect to A1
# _Z2 = A1 * W2 + B2
# dZ2_dA1 = diff(_Z2, A1)
# print("dZ2_dA1 =", dZ2_dA1)

# # derivative of Z2 with respect to W2
# dZ2_dW2 = diff(_Z2, W2)
# print("dZ2_dW2 =", dZ2_dW2)

# # derivative of Z2 with respect to B2
# dZ2_dB2 = diff(_Z2, B2)
# print("dZ2_dB2 =", dZ2_dB2)

# # derivative of A1 with respect to Z1
# der_relu = lambda x: Max(x,0)
# _A1 = der_relu(Z1)
# d_relu = lambda x: x > 0
# dA1_dZ1 = d_relu(Z1)
# print("dA1_dZ1 =", dA1_dZ1)

# # derivative of Z1 with respect to W1
# _Z1 = W1 * X + B1
# dZ1_dW1 = diff(_Z1, W1)
# print("dZ1_dW1 =", dZ1_dW1) 

# # derivative of Z1 with respect to B1
# dZ1_dB1 = diff(_Z1, B1)
# print("dZ1_dB1 =", dZ1_dB1)

dC_dA2 = 2*A2 - 2*Y
dA2_dZ2 = exp(-Z2)/(1 + exp(-Z2))**2
dZ2_dA1 = W2
dZ2_dW2 = A1
dZ2_dB2 = 1
dA1_dZ1 = Z1 > 0
dZ1_dW1 = X
dZ1_dB1 = 1


In [5]:
# Backpropagtion method for weights and biases using chain rule for find W1, B1, W2 and B2

# derivatives of activation functions
deriv_relu = lambda x: (x > 0).astype(float)
deriv_sigmoid = lambda x: np.exp(-x) / (1 + np.exp(-x))**2

def backpropagation(Z1, A1, Z2, A2, X, Y):
    dC_dA2 = 2*A2 - 2*Y
    dA2_dZ2 = deriv_sigmoid(Z2)
    dZ2_dA1 = w_output
    dZ2_dW2 = A1
    dZ2_dB2 = 1
    dA1_dZ1 = deriv_relu(Z1)
    dZ1_dW1 = X
    dZ1_dB1 = 1

    dC_dW2 = dC_dA2 @ dA2_dZ2 @ dZ2_dW2.T
    dC_dB2 = dC_dA2 @ dA2_dZ2 * dZ2_dB2
    dC_dA1 = dC_dA2 @ dA2_dZ2 @ dZ2_dA1
    dC_dW1 = dC_dA1 @ dA1_dZ1 @ dZ1_dW1.T
    dC_dB1 = dC_dA1 @ dA1_dZ1 * dZ1_dB1

    return dC_dW1, dC_dB1, dC_dW2, dC_dB2

# Stochastic gradient optimaization
for i in range(10000):
    # choosing one training data 
    idx = np.random.choice(n, 1, replace=False)
    X_sample = X_train[idx].transpose()
    Y_sample = Y_train[idx]

    # Run random data(sample) through a neural network
    Z1, A1, Z2, A2 = forward_pass(X_sample)

    # Optimazation neural network using backpropagation
    dW1, dB1, dW2, dB2 = backpropagation(Z1, A1, Z2, A2, X_sample, Y_sample)

    # Update of weights and biases
    w_hidden -= L * dW1
    w_output -= L * dW2
    b_hidden -= L * dB1
    b_output -= L * dB2

# Checking the ACCURACY of neural network
prediction = forward_pass(X_test.transpose())[3] # take only A2
comparison = np.equal((prediction >= .5).flatten().astype(int), Y_test)
accuracy = sum(comparison.astype(int) / X_test.shape[0])
print("ACCURACY: ", accuracy)

ACCURACY:  0.9910913140311729


In [ ]:
# Testing the font color for the indicated background color 
# (dark font for a light background, and light font for a dark background).

def pred_of_probability(r, g, b):
    X = np.array([[r, g, b]]).transpose() / 255
    Z1, A1, Z2, A2 = forward_pass(X)
    return A2

def pred_color_of_font(r, g, b):
    value_of_probability = pred_of_probability(r, g, b)
    if value_of_probability > .5:
        return "DARK FONT"
    else:
        return "LIGHT FONT"

while True:
    color_of_background = input("Light or Dark font? The network will match the shade of the font color to the background color.\
Enter comma-separated values of R (number from 0 to 255), G (number from 0 to 255), B (number from 0 to 255) to determine the background color.: ")
    (r, g, b) = color_of_background.split(",")
    print(pred_color_of_font(int(r), int(g), int(b)))

Light or Dark font? The network will match the shade of the font color to the background color.Enter comma-separated values of R (number from 0 to 255), G (number from 0 to 255), B (number from 0 to 255) to determine the background color.:  255,192,203


DARK FONT


Light or Dark font? The network will match the shade of the font color to the background color.Enter comma-separated values of R (number from 0 to 255), G (number from 0 to 255), B (number from 0 to 255) to determine the background color.:  0,0,0


LIGHT FONT


Light or Dark font? The network will match the shade of the font color to the background color.Enter comma-separated values of R (number from 0 to 255), G (number from 0 to 255), B (number from 0 to 255) to determine the background color.:  255,140,0


DARK FONT


Light or Dark font? The network will match the shade of the font color to the background color.Enter comma-separated values of R (number from 0 to 255), G (number from 0 to 255), B (number from 0 to 255) to determine the background color.:  255,255,255


DARK FONT


Light or Dark font? The network will match the shade of the font color to the background color.Enter comma-separated values of R (number from 0 to 255), G (number from 0 to 255), B (number from 0 to 255) to determine the background color.:  0,0,139


LIGHT FONT
